In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from torch import nn
from pandas import DataFrame as DF
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pylab as plt
from skimage import io, transform
from torchvision import transforms
from albumentations import (HorizontalFlip, VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)

In [3]:
dirct = '../input/sartorius-cell-instance-segmentation'

traindf = pd.read_csv(dirct + "/train.csv")
traindf.head(1)

In [4]:
imagename = traindf['id'].unique()[0]
dfimage = traindf[traindf['id']==imagename]
annota_all = dfimage['annotation']
print(annota_all[2])

image = io.imread(dirct + '/train/' + imagename + '.png') 
plt.imshow(image)
print(image.shape)

In [8]:
annot = rle_decode(annota_all[2], image.shape)
print(annot)
plt.imshow(annot)

In [6]:
def rle_decode(rle_annota, shape, color = 1):
    s = rle_annota.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    for i in range(len(starts)):
        lo = starts[i]
        hi = ends[i]
        img[lo:hi] = color
    return img.reshape(shape)

In [7]:
"decode mask for a certain image"
def create_mask(df_train, img_id, shape):
    dfimage = df_train[df_train['id'] == img_id]
    annota_all = dfimage['annotation'].tolist()
    img_mask = np.zeros(shape)
    for i in range(len(annota_all)):
        mask_i = rle_decode(annota_all[i], shape)
        img_mask += mask_i
    img_mask = img_mask.clip(0, 1)
    return np.array(img_mask)

In [9]:
a = create_mask(traindf, imagename, image.shape)
plt.imshow(a)

In [17]:
class Data(Dataset):
    def __init__(self, df: pd.core.frame.DataFrame, train_pct, train = bool):
        self.Image = (224, 224)
        self.RESNET_MEAN = (0.485, 0.456, 0.406)
        self.RESNET_STD = (0.229, 0.224, 0.225)
        self.df = df
        self.dir = data_dir['TRAIN_PATH']
        self.gb = self.df.groupby('id')
 #       self.trans = transforms.Compose([Resize(self.Image), ToTensor()])
#         self.trans = transforms.Compose([transforms.ToPILImage(),
#                                          transforms.Resize(self.Image),
#                                          transforms.RandomHorizontalFlip(),
#                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                          std=[0.229, 0.224, 0.225]),
#                                          transforms.ToTensor()])
        self.trans = Compose([Resize(self.Image[0], self.Image[1]), 
                                   Normalize(mean=self.RESNET_MEAN[0], std= self.RESNET_STD[0], p=1), 
                                   HorizontalFlip(p=0.5),
                                   VerticalFlip(p=0.5)])
        all_img = np.array(df.id.unique())  #create an array for all image id
        np.random.seed(42)
        idx_shuf = np.random.permutation(len(all_img))  #shuffle the dataset
        num_train = int(len(all_img)*train_pct) #percent of training set
        if train:
            self.img_sf = all_img[idx_shuf[:num_train]]
        else:
            self.img_sf = all_img[idx_shuf[num_train:]]
            
    def __len__(self):
        return len(self.img_sf)
    
    #get image sample with idx from the dataset for each item
    def __getitem__(self, idx):
        img_samp = self.img_sf[idx]
        dfimage = self.gb.get_group(img_samp)#traindf[traindf['id']==img_samp] #each id has lots of img, group them
        image_path = os.path.join(self.dir, img_samp) + ".png"
        
        image = io.imread(image_path)
        mask = create_mask(self.df, img_samp, image.shape)
        mask = (mask >= 1).astype('float32')
        augmented = self.trans(image=image, mask=mask)
        img_aug = augmented['image']
        msk_aug = augmented['mask']
        img_aug = img_aug.astype('float32')
        return img_aug.reshape((1, self.Image[0], self.Image[1])), msk_aug.reshape((1, self.Image[0], self.Image[1]))

In [13]:
"specify data directories, join means add \with file names"
DATA_PATH = '../input/sartorius-cell-instance-segmentation'
data_dir = { 'SAMPLE_SUBMISSION': os.path.join(DATA_PATH, 'train'),
             'TRAIN_CSV': os.path.join(DATA_PATH, 'train.csv'),
             'TRAIN_PATH': os.path.join(DATA_PATH, 'train'),
             'TEST_PATH': os.path.join(DATA_PATH, 'test')}
print(data_dir['TRAIN_CSV'])

In [18]:
img_train = Data(pd.read_csv(data_dir['TRAIN_CSV']), 0.9, train=True) 
img_load  = DataLoader(img_train, batch_size=16, num_workers=2, pin_memory=True, shuffle=False)

In [19]:
img_test = Data(pd.read_csv(data_dir['TRAIN_CSV']), 0.9, train=False) #traindf def early on top
test_load = DataLoader(img_test, batch_size=4, num_workers=2, pin_memory=True, shuffle=False)

In [20]:
batch = next(iter(test_load))
images, masks = batch #first minibatch
print(f"image shape: {images.shape},\nmask shape:{masks.shape},\nbatch len: {len(batch)}")
print(images.dtype, masks.dtype)
plt.figure(figsize=(20, 20))
        
plt.subplot(1, 3, 1)
plt.xticks([])
plt.yticks([])
plt.imshow(images[1][0])
plt.title('Original image')

plt.subplot(1, 3, 2)
plt.xticks([])
plt.yticks([])
plt.imshow(masks[1][0])
plt.title('Mask')

plt.subplot(1, 3, 3)
plt.xticks([])
plt.yticks([])
plt.imshow(images[1][0])
plt.imshow(masks[1][0],alpha=0.2)
plt.title('Both')
plt.tight_layout()
plt.show()

In [21]:
print(masks[1])

In [22]:
print("whole batch",len(img_train), "num of minibatch", len(img_load), "with batch size 16")

In [23]:
"construct U-Net from scratch"
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch, kernel):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential( 
            nn.Conv2d(in_ch, out_ch, kernel_size = kernel, stride=1, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size = kernel, stride=1, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
         )
    def forward(self, x):
        x = self.conv(x)
        return x

In [25]:
class Down(nn.Module):
    def __init__(self, in_ch, out_ch, kernel):
        super().__init__()
        self.down = nn.Sequential(nn.MaxPool2d(2,2),
                                 DoubleConv(in_ch, out_ch, kernel))   #how Down class inherits DoubleConv?
        
    def forward(self, x):
        x = self.down(x)
        return x
    
class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch //2, in_ch //2, kernel_size = 2, stride = 2)
        self.conv = DoubleConv(in_ch, out_ch, 3) 
        
    def forward(self, feature, context):
        x = self.up(feature)
        
        skip = torch.cat([x, context], dim = 1)   #concatenate context&feature map
        up_x = self.conv(skip)          
        return up_x

class out_layer(nn.Module):
    def __init__(self, in_ch, num_class):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, num_class, 1) #final layer by 1x1 conv
        self.sigmoid = nn.Sigmoid() #elementwise calculation of sigmoid, each layer (class)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.sigmoid(x)
        return x  

In [28]:
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(UNet, self).__init__()
        self.inc = DoubleConv(in_channels, 64, 3)
        self.down1 = Down(64, 128, 3)
        self.down2 = Down(128, 256, 3)
        self.down3 = Down(256, 512, 3)
        self.down4 = Down(512, 512, 3)
        self.up1 = Up(1024, 256)
        self.up2 = Up(512, 128)
        self.up3 = Up(256, 64)
        self.up4 = Up(128, 64)
        self.outc = out_layer(64, num_classes)  #out layer w/ different classes, each channel w/ binary label
    def forward(self, x):
        #print(x)
        x1 = self.inc(x)
        #print(x1)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

In [26]:
def DiceLoss(inputs, targets, smooth=1):
        
    #comment out if your model contains a sigmoid or equivalent activation layer
    #inputs = F.sigmoid(inputs)       
        
    #flatten label and prediction tensors
    inputs = inputs.view(-1)
    targets = targets.view(-1)
        
    intersection = (inputs * targets).sum()                            
    dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
    return 1 - dice

In [29]:
model = UNet(1, 1) #input single channel and output single layer for one class

In [30]:
a = model(images[:3])
print(a)

In [31]:
b = masks[:3]
print(b)
print(b.shape)
l = DiceLoss(a, b)
print(l)

In [32]:
def train(model, data_l, learning_rate):
    cost = []
    for img, msk in data_l:
        optimizer.zero_grad()
        out = model(img.to(device))
        loss = DiceLoss(out, msk.to(device))
        loss.backward()
        optimizer.step()
        cost.append(loss.item())

    return cost

def eval_loop(model, optimizer, eval_l):  #evaluate the test set
    running_loss = 0
    model.eval()
    with torch.no_grad():
        f1_scores, accuracy = [], []
        for imgs, masks in eval_l:
            # pass to device
            imgs = imgs.to(device)
            masks = masks.to(device)
            # forward
            out = model(imgs)
            loss = DiceLoss(out, masks)
            running_loss += loss.item()*imgs.shape[0]
            # calculate predictions using output
            predicted = (out > 0.5).float()  
            predicted = predicted.view(-1).cpu().numpy()   #turn tensor to array for accuracy
            print(predicted)
            masks = (masks > 0.5).float()  
            labels = masks.view(-1).cpu().numpy()
            print(labels)
            accuracy.append(accuracy_score(labels, predicted))
            f1_scores.append(f1_score(labels, predicted))
    acc = sum(accuracy)/len(accuracy)
    f1 = sum(f1_scores)/len(f1_scores)
    running_loss /= len(img_test)
    return acc, f1, running_loss

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
device = torch.device('cpu')

In [58]:
"test run with 10 samples"
import itertools
cost = []
for img, msk in itertools.islice(img_load, 10):
    out = model(img.to(device))
    loss = DiceLoss(out, msk.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    cost.append(loss.item())

In [38]:
cost_list = []
num_epoch = 5
for e in range(num_epoch):
    mini_cost = train(model, img_load, 0.01)
    cost_list += mini_cost

In [39]:
plt.plot(cost_list)
plt.show()

In [40]:
acc_tot, f_tot, loss_tot = eval_loop(model, optimizer, test_load)

In [44]:
print(acc_tot, f_tot, loss_tot)

In [41]:
with torch.no_grad():
    outputs = []
    ground_trues = []
    imgs_test = []
    for imgs, masks in test_load:
        # pass to device
        imgs = imgs.to(device)
        masks = masks.to(device)
        # forward
        out = model(imgs)
        predicted = (out > 0.5).float()
#         print(predicted.shape)
        outputs.extend(predicted)
        ground_trues.extend(masks)
        imgs_test.extend(imgs) 

In [42]:
for i in range(5):
    plt.figure(figsize=(8, 5))
        
    plt.subplot(1, 3, 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imgs_test[i][0].cpu().numpy())
    plt.title('Original image')

    plt.subplot( 1, 3, 2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(ground_trues[i][0].cpu().numpy())
    plt.title('Mask')


    plt.subplot(1, 3, 3)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(outputs[i][0].cpu().numpy())
    plt.title('Predicted')
    plt.tight_layout()
    plt.show()